In [141]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import seaborn as sns
sns.set()

pd.set_option('display.float_format', '{:.2f}'.format)

In [216]:
#Importation du fichier population
df_population = pd.read_csv("Population_2000_2018.csv", delimiter=';')

#Importation du fichier dispo alimentaire
df_disponibilitealimentaire = pd.read_csv("DisponibiliteAlimentaire_2017.csv", delimiter=';')

#Importation du fichier production de  poulet
df_infopoulet = pd.read_csv("production_poulet.csv", delimiter=';')

#Importation du fichier stabilité politique
df_PoliticalStability = pd.read_csv("PoliticalStability.csv", delimiter=';')

#Importation du fichier stabilité politique
df_qualite_vie = pd.read_csv("qualite_vie.csv", delimiter=';')

#Importation du fichier import export
df_imp_exp = pd.read_csv("données_import_export.csv", delimiter=';')
df = df_imp_exp

# Créer une colonne 'Importation_quantité' avec des valeurs vides
df['Importation_quantité'] = ''
# Créer une colonne 'Importation_valeur' avec des valeurs vides
df['Importation_valeur'] = ''
# Créer une colonne 'Exportation_quantité' avec des valeurs vides
df['Exportation_quantité'] = ''
# Créer une colonne 'Exportation_valeur' avec des valeurs vides
df['Exportation_valeur'] = ''

df_pivot = df.pivot_table(index=['Zone', 'Produit', 'Année'], columns='Élément', values=['Valeur'], aggfunc='first').reset_index()
df_pivot.df_import_export = ['Zone', 'Produit', 'Année', 'Exportation_quantité', 'Exportation_valeur', 'Importation_quantité', 'Importation_valeur']
# Afficher le résultat
df_import_export = df_pivot
#df_population = df_population.groupby('Zone').mean().reset_index()
#df_population = df_population.round(0)
#df_infopoulet = df_infopoulet.groupby('Zone').mean().reset_index()
#df_infopoulet = df_infopoulet.round(0)
#df_PoliticalStability = df_PoliticalStability.groupby('Zone').mean().reset_index()
#df_import_export = df_import_export.groupby('Zone').mean().reset_index()
#df_import_export = df_import_export.round(0)
display(df_infopoulet)


<ipython-input-216-c86df4981f36>:30: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_pivot.df_import_export = ['Zone', 'Produit', 'Année', 'Exportation_quantité', 'Exportation_valeur', 'Importation_quantité', 'Importation_valeur']


,Zone,Année,Valeur
0,Afghanistan,2002,12156
1,Afghanistan,2003,12402
2,Afghanistan,2004,13022
3,Afghanistan,2005,14414
4,Afghanistan,2006,10880
...,...,...,...
4068,Zimbabwe,2018,14893
4069,Zimbabwe,2019,17000
4070,Zimbabwe,2020,16200
4071,Zimbabwe,2021,15583


In [217]:
#Fusion des fichiers df_population et df_PoliticalStability
df_merge_tmp = pd.merge(df_population, df_PoliticalStability, on=['Zone', 'Année'], how='inner')
#Suprresion des colonnes inutiles
#modification des noms des colonnes
df_merge_tmp.rename(columns={'Valeur_x': 'valeur_population'}, inplace=True)
df_merge_tmp.rename(columns={'Valeur_y': 'stabilite'}, inplace=True)
df_merge_tmp.drop(['Élément','Produit','Unité','Description du Symbole','Note'], axis=1, inplace=True)
df_merge_tmp = df_merge_tmp.fillna(0)
#Fusion des fichiers df_merge_tmp et df_infopoulet 
df_merge_tmp1 = pd.merge(df_merge_tmp,df_infopoulet, on=['Zone', 'Année'], how='left')
#Suppresion des colonnes inutiles
#df_merge_tmp1.drop(['Domaine','Élément'], axis=1, inplace=True)
display(df_merge_tmp1)
#modification des noms des colonnes
#df_merge_tmp1.rename(columns={'Unité': 'unite_production'}, inplace=True)
#df_merge_tmp1.rename(columns={'Valeur': 'valeur_production'}, inplace=True)
#df_merge_tmp1.drop(['Domaine','Élément','Produit'], axis=1, inplace=True)
df_merge_tmp1 = df_merge_tmp1.fillna(0)




#Fusion des fichiers df_merge_tmp1 et df_import_export
df_merge = pd.merge(df_merge_tmp1,df_import_export, on=['Zone', 'Année'])
#df_merge.drop(['Année','unite_production''unite_production''unite_production''unite_production'], axis=1, inplace=True)
display(df_merge)

#modification des noms des colonnes
print("Le tableau comporte {} observation(s) ou article(s)".format(df_merge.shape[0]))
print("Le tableau comporte {} colonne(s)".format(df_merge.shape[1]))
df_merge.columns = ['Zone', 'Année','valeur_population', 'stabilite', 'valeur_production','Produit',  'Exportations - Quantité', 'Exportations - Valeur', 'Importations - Quantité', 'Importations - Valeur']
df_merge = df_merge.fillna(0)
df_merge.drop(['Année','Produit'], axis=1, inplace=True)
df_merge['stabilite'] = df_merge['stabilite'].str.replace(',', '.').astype(float)
df_merge =df_merge.groupby('Zone').mean()
df_merge = pd.merge(df_merge,df_qualite_vie, on=['Zone'])
display(df_merge)

,Zone,Année,valeur_population,stabilite,Valeur
0,Afghanistan,2002,22600.77,"-2,04",12156.00
1,Afghanistan,2003,23680.87,"-2,20",12402.00
2,Afghanistan,2004,24726.68,"-2,30",13022.00
3,Afghanistan,2005,25654.28,"-2,07",14414.00
4,Afghanistan,2006,26433.05,"-2,22",10880.00
...,...,...,...,...,...
3265,Zimbabwe,2014,13586.71,"-0,71",13311.00
3266,Zimbabwe,2015,13814.63,"-0,62",14278.00
3267,Zimbabwe,2016,14030.33,"-0,62",14849.00
3268,Zimbabwe,2017,14236.59,"-0,71",15551.00


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,Zone,Année,valeur_population,stabilite,Valeur,"(Produit, )","(Valeur, Exportations - Quantité)","(Valeur, Exportations - Valeur)","(Valeur, Importations - Quantité)","(Valeur, Importations - Valeur)"
0,Afghanistan,2003,23680.87,"-2,20",12402.00,Poulets,14.00,33.00,2400.00,3741.00
1,Afghanistan,2004,24726.68,"-2,30",13022.00,Poulets,0.00,0.00,3023.00,3372.00
2,Afghanistan,2005,25654.28,"-2,07",14414.00,Poulets,0.00,0.00,2411.00,2211.00
3,Afghanistan,2006,26433.05,"-2,22",10880.00,Poulets,0.00,0.00,6827.00,1855.00
4,Afghanistan,2007,27100.54,"-2,41",9035.00,Poulets,0.00,0.00,81.00,197.00
...,...,...,...,...,...,...,...,...,...,...
2794,Zimbabwe,2014,13586.71,"-0,71",13311.00,Poulets,1844.00,2904.00,370.00,2264.00
2795,Zimbabwe,2015,13814.63,"-0,62",14278.00,Poulets,2031.00,3034.00,684.00,2792.00
2796,Zimbabwe,2016,14030.33,"-0,62",14849.00,Poulets,766.00,1924.00,367.00,2313.00
2797,Zimbabwe,2017,14236.59,"-0,71",15551.00,Poulets,211.00,653.00,403.00,2064.00


Le tableau comporte 2799 observation(s) ou article(s)
Le tableau comporte 10 colonne(s)


,Zone,valeur_population,stabilite,valeur_production,Exportations - Quantité,Exportations - Valeur,Importations - Quantité,Importations - Valeur,indice_qualite_vie
0,Afghanistan,30192.62,-2.49,12072.75,1.38,2.31,5409.62,5188.94,34
1,Afrique du Sud,51902.38,-0.09,163444.12,3179.25,3722.81,536.50,7346.88,40
2,Albanie,2969.82,-0.04,6523.75,11.56,7.88,5343.00,3553.75,64
3,Algérie,36685.13,-1.18,128543.19,0.00,0.00,3623.62,16248.50,54
4,Allemagne,81538.96,0.80,116996.12,216731.12,468504.06,147424.62,123145.12,85
...,...,...,...,...,...,...,...,...,...
110,Zambie,14017.04,0.32,34923.50,526.50,1556.12,413.56,1961.06,51
111,Zimbabwe,12972.59,-0.93,21573.12,1125.38,1390.06,308.44,1880.06,38
112,Égypte,84543.52,-1.08,113612.56,1972.25,1058.62,2348.50,1802.50,48
113,Équateur,15151.21,-0.51,146760.69,29.06,103.38,2712.38,9897.00,61


In [218]:
#%age de valeur manquante
df_merge.isnull().sum()
df_merge.isnull().sum()

Zone                       0
valeur_population          0
stabilite                  0
valeur_production          0
Exportations - Quantité    0
Exportations - Valeur      0
Importations - Quantité    0
Importations - Valeur      0
indice_qualite_vie         0
dtype: int64

In [220]:
#Doublons
df_merge.duplicated().sum()

0

In [221]:
# differente valeur par colonne
df_merge.nunique()


Zone                       115
valeur_population          115
stabilite                  115
valeur_production          115
Exportations - Quantité    100
Exportations - Valeur      104
Importations - Quantité    113
Importations - Valeur      113
indice_qualite_vie          49
dtype: int64

In [190]:
#Multiplier les valeur xx par 1000
df_merge['valeur_population'] = df_merge['valeur_population'] * 1000
df_merge['valeur_production'] = df_merge['valeur_production'] * 1000
df_merge['Exportations - Quantité'] = df_merge['Exportations - Quantité'] * 1000
df_merge['Exportations - Valeur'] = df_merge['Exportations - Valeur'] * 1000
df_merge['Importations - Quantité'] = df_merge['Importations - Quantité'] * 1000
df_merge['Importations - Valeur'] = df_merge['Importations - Valeur'] * 1000

#Afficher les 5 premières lignes de la table
display(df_merge.head(5))

,Zone,valeur_population,stabilite,valeur_production,Exportations - Quantité,Exportations - Valeur,Importations - Quantité,Importations - Valeur,indice_qualite_vie
0,Afghanistan,30192615.88,-2.49,12072750.00,1375.00,2312.50,5409625.00,5188937.50,34
1,Afrique du Sud,51902379.87,-0.09,163444125.00,3179250.00,3722812.50,536500.00,7346875.00,40
2,Albanie,2969821.00,-0.04,6523750.00,11562.50,7875.00,5343000.00,3553750.00,64
3,Algérie,36685133.81,-1.18,128543187.50,0.00,0.00,3623625.00,16248500.00,54
4,Allemagne,81538961.06,0.80,116996125.00,216731125.00,468504062.50,147424625.00,123145125.00,85


In [222]:
df_merge.describe()


,valeur_population,stabilite,valeur_production,Exportations - Quantité,Exportations - Valeur,Importations - Quantité,Importations - Valeur,indice_qualite_vie
count,115.00,115.00,115.00,115.00,115.00,115.00,115.00,115.00
mean,37134.26,-0.12,93449.64,8054.63,11287.00,7819.02,10967.60,60.95
std,120021.86,0.91,220380.96,25925.92,48571.49,22245.44,30312.22,16.63
min,309.32,-2.49,0.00,0.00,0.00,0.00,0.00,23.00
25%,4765.21,-0.78,9064.16,7.72,5.69,325.91,883.16,48.50
50%,10702.70,-0.09,24919.81,157.81,221.56,1055.75,3396.62,61.00
75%,29757.12,0.70,97074.00,2549.81,3194.47,3710.19,8608.91,75.00
max,1238039.33,1.39,1756560.38,216731.12,468504.06,150123.94,268680.38,89.00


In [192]:
print(df_merge.index.to_list())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114]


In [223]:
display(df_merge.dtypes)
df_merge = df_merge.set_index('Zone')

cols = ["valeur_population", "stabilite", "valeur_production", "Exportations - Quantité", "Exportations - Valeur", "Importations - Quantité", "Importations - Valeur", "indice_qualite_vie"]
df_merge = df_merge[cols]


Zone                        object
valeur_population          float64
stabilite                  float64
valeur_production          float64
Exportations - Quantité    float64
Exportations - Valeur      float64
Importations - Quantité    float64
Importations - Valeur      float64
indice_qualite_vie           int64
dtype: object

In [224]:
df_merge.shape

(115, 8)

In [225]:
# Afficher les lignes avec des valeurs nulles dans n'importe quelle colonne
df_null = df_merge[df_merge.isnull().any(axis=1)]

display(df_null)

,valeur_population,stabilite,valeur_production,Exportations - Quantité,Exportations - Valeur,Importations - Quantité,Importations - Valeur,indice_qualite_vie
Zone,,,,,,,,


In [227]:
df_merge.to_csv('Donnees_preparees.csv', index=True)